In [1]:
from dotenv import load_dotenv
from datasets import load_dataset,Dataset
import os
from huggingface_hub import login
import sqlite3

In [2]:
load_dotenv()
hf_token = os.getenv("HF_TOKEN")

login(hf_token,add_to_git_credential=True)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
raw_train_data = load_dataset("chrisjcc/text-to-sql-spider-dataset", split="train",trust_remote_code = True)


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'chrisjcc/text-to-sql-spider-dataset' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


In [4]:
raw_train_data

Dataset({
    features: ['messages'],
    num_rows: 20000
})

In [5]:
raw_train_data[0]


{'messages': [{'content': 'You are a text-to-SQL query translator. Users will ask questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE medicine (id VARCHAR, name VARCHAR); CREATE TABLE medicine_enzyme_interaction (enzyme_id VARCHAR, medicine_id VARCHAR, interaction_type VARCHAR); CREATE TABLE enzyme (name VARCHAR, id VARCHAR)',
   'role': 'system'},
  {'content': "What are the names of enzymes in the medicine named 'Amisulpride' that can serve as an 'inhibitor'?",
   'role': 'user'},
  {'content': "SELECT T1.name FROM enzyme AS T1 JOIN medicine_enzyme_interaction AS T2 ON T1.id = T2.enzyme_id JOIN medicine AS T3 ON T2.medicine_id = T3.id WHERE T3.name = 'Amisulpride' AND T2.interaction_type = 'inhibitor'",
   'role': 'assistant'}]}

In [6]:
filtered_train = (data for data in raw_train_data 
            if 'messages' in data and data['messages'][0]['content'] is not None and data['messages'][0]['content'].strip() != ""
            and data['messages'][1]['content'] is not None and data['messages'][1]['content'].strip()!= ""
            and data['messages'][2]['content'] is not None)


In [7]:
sample_train_data = []
for dt in filtered_train:
    sample_train_data.append(dt)

dataset_train = Dataset.from_list(sample_train_data)
print(f"Loaded: {len(dataset_train)}")


Loaded: 20000


In [8]:
dataset_train[0]['messages'][0]['content']

'You are a text-to-SQL query translator. Users will ask questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE medicine (id VARCHAR, name VARCHAR); CREATE TABLE medicine_enzyme_interaction (enzyme_id VARCHAR, medicine_id VARCHAR, interaction_type VARCHAR); CREATE TABLE enzyme (name VARCHAR, id VARCHAR)'

In [9]:
%pwd

'e:\\work\\Finnetune\\Model_Text2SQL\\research'

In [10]:
os.chdir("E:\work\Finnetune\Model_Text2SQL")

In [11]:
from src.DataItem import DataItemt

dataset = dataset_train.shuffle(seed=14)
trains = []
tests = []
vals = []
test_value = []
for i,data in enumerate(dataset):
    try:
        data = DataItemt(data)
        if i < 16000:
            data.MakePromt()
            trains.append(data.prompt)
        elif 16000<= i <18000:
            data.MakePromt()
            vals.append(data.prompt)
        else:
            tests.append(data.TestPrompt())
            test_value.append(data.TestValue())
    except(ValueError,TypeError):
        pass


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [12]:
print(len(trains))
print(len(tests))
print(len(vals))
print(len(test_value))

16000
2000
2000
2000


In [13]:
trains[0]

'You are a text-to-SQL query translator. Users will ask questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE table_name_49 (date VARCHAR, opponent VARCHAR, save VARCHAR)\nUSER:\nWhat is the date of the match with Brother Elephants as the opponent and a save of ||4,117?\n\nOutput:\nQUERY:\nSELECT date FROM table_name_49 WHERE opponent = "brother elephants" AND save = "||4,117"'

In [14]:
tests[0]

'You are a text-to-SQL query translator. Users will ask questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE Accounts (statement_id VARCHAR); CREATE TABLE Statements (statement_details VARCHAR, statement_id VARCHAR)\nUSER:\nShow the statement id and the statement detail for the statement with most number of accounts.\n\nOutput:\nQUERY:\n'

In [15]:
test_value[0]

'SELECT T1.statement_id, T2.statement_details FROM Accounts AS T1 JOIN Statements AS T2 ON T1.statement_id = T2.statement_id GROUP BY T1.statement_id ORDER BY COUNT(*) DESC LIMIT 1'

In [16]:
from datasets import DatasetDict

train_dataset = Dataset.from_dict({"text":trains})
test_dataset = Dataset.from_dict({"text":tests})
val_dataset = Dataset.from_dict({"text":vals})
test_value_dataset = Dataset.from_dict({"text":test_value})

dataset = DatasetDict({
    "train":train_dataset,
    "test":test_dataset,
    "val":val_dataset,
    "test_value":test_value_dataset
})


In [17]:
os.environ.pop("HF_TOKEN", None)
hf_token_write = os.getenv('HF_TOKEN_WRITE')
login(hf_token_write,add_to_git_credential=True)

In [18]:
HF_USER = "Hoangee2"
DATASET_NAME = f"{HF_USER}/Text2SQL"
dataset.push_to_hub(DATASET_NAME, private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

...

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

e:\work\Finnetune\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\datasets--Hoangee2--Text2SQL. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/Hoangee2/Text2SQL/commit/f94456c28b42fc6325f8524a49ef74f442528648', commit_message='Upload dataset', commit_description='', oid='f94456c28b42fc6325f8524a49ef74f442528648', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Hoangee2/Text2SQL', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Hoangee2/Text2SQL'), pr_revision=None, pr_num=None)

In [19]:
import pickle

with open('train.pkl','wb') as file:
    pickle.dump(trains,file)

with open('test.pkl','wb') as file:
    pickle.dump(tests,file)

with open('val.pkl','wb') as file:
    pickle.dump(vals,file)

with open('test_value.pkl','wb') as file:
    pickle.dump(test_value,file)